In [1]:
import os
import paramiko
import scp

from HPC_MD.Lumi_Interface import HPCConnection
from HPC_MD.Call_Runner import generate_md_runner

In [ ]:
hpc_connector = HPCConnection(key_passphrase="Dm761pm*")
hpc_connector.connect()

In [3]:
remote_dir = "/scratch/project_465001750/LAG3_V2/DIOCANE"

In [ ]:
hpc_connector.download_and_rename_directory(remote_dir, "./", "LLERO")

In [ ]:
custom_config = {
    "mode": {"type": "legacy"},
    "legacy": {
        "runs": {
            "Run1": {"protein": "./protein1.pdb", "ligand": "./ligand1.sdf"},
            "Run2": {"protein": "./protein2.pdb", "ligand": "APO"}
        }
    }
}

runner = generate_md_runner(
    account="project_465001750",
    toml_config=custom_config
)
print(runner)